# Pipeline Maestro - Orquestador de Notebooks

Este notebook define todas las configuraciones centralizadas y ejecuta el pipeline completo de notebooks en orden.

## Instrucciones de Uso

### Opción 1: Ejecución Automatizada (Recomendada)

1. **Configurar fechas** en la celda "Configuración Central de Fechas"
2. **Configurar notebooks a ejecutar** en la celda "Configuración de Ejecución"
3. **Ejecutar todas las celdas** hasta "Ejecución del Pipeline Completo"
4. **Para NB07 (Neural Networks)**:
   - Si `EJECUTAR_NB07_EN_COLAB = True`: El pipeline se detendrá automáticamente
   - Sigue las instrucciones para ejecutar NB07 en Google Colab con GPU
   - Descarga los resultados y cópialos a la carpeta `artifacts/`
   - Ejecuta la celda "Verificación de Resultados de NB07"
   - Ejecuta la celda "Continuación del Pipeline" para NB08-10
5. **Revisar resultados** en la última celda

### Opción 2: Ejecución Manual

1. **Abrir cada notebook individualmente** (01-10)
2. **Modificar las fechas** en las celdas de parámetros de cada notebook
3. **Ejecutar todas las celdas** de cada notebook en orden
4. **NB07 opcional**: Ejecutar en Google Colab para aprovechar GPU

### Opción 3: Ejecución Híbrida

1. **Ejecutar algunos notebooks manualmente** para exploración
2. **Usar este NB00** para automatizar el resto del pipeline

## Configuración Centralizada

Define aquí una sola vez:
- **Rango de fechas para generación de datos** (NB01, NB02)
- **Fecha de corte Train/Validation** (NB03)
- **Rango de fechas para entrenamiento** (NB04, NB05, NB06, NB07)

## Pipeline de Ejecución

1. **NB01** - Data Preparation: Carga y preprocesa datos de fuentes
2. **NB02** - Feature Engineering: Crea características temporales y lags
3. **NB03** - Exploratory Analysis: Separa train/validation, selecciona features
4. **NB04** - Baseline Models: Entrena modelos básicos de referencia
5. **NB05** - Machine Learning: Entrena modelos tree-based con hiperparámetros manuales
6. **NB06** - Hyperparameter Optimization: Optimiza con Optuna (opcional)
7. **NB07** - Neural Networks: Entrena redes neuronales (MLP, LSTM, CNN-LSTM) - **RECOMENDADO EN GOOGLE COLAB CON GPU**
8. **NB08** - Model Comparison: Compara todos los modelos
9. **NB09** - Model Validation: Valida modelo ganador en datos futuros
10. **NB10** - Production Validation: Valida con datos actualizados de producción

## Ventajas

- Configuración única y centralizada
- Ejecución automatizada de todo el pipeline
- Consistencia en fechas entre todos los notebooks
- Fácil modificación de parámetros
- Reproducibilidad completa

In [1]:
# Importar bibliotecas necesarias
import pandas as pd
from pathlib import Path
from datetime import datetime
import json
import warnings

warnings.filterwarnings('ignore')

print("=" * 80)
print("PREDICCIÓN DE DEMANDA ELÉCTRICA")
print("=" * 80)
print(f"\nFecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

PREDICCIÓN DE DEMANDA ELÉCTRICA

Fecha de ejecución: 2025-10-26 19:45:02


## Gestión de Backup de Artifacts

**OPCIONAL**: Guarda el contenido actual de `artifacts/` antes de ejecutar el pipeline.

In [2]:
# ============================================================================
# CONFIGURACIÓN DE BACKUP
# ============================================================================

# Cambiar a True si quieres hacer backup del artifacts actual antes de ejecutar
# True para guardar backup, False para no hacer nada
HACER_BACKUP_ARTIFACTS = True

# ============================================================================
# GESTIÓN AUTOMÁTICA DE BACKUP
# ============================================================================

if HACER_BACKUP_ARTIFACTS:
    import shutil
    from pathlib import Path
    from datetime import datetime
    
    # Crear nombre de carpeta con fecha y hora actual
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    backup_dir = Path('../../.backup_pipeline') / timestamp
    artifacts_dir = Path('artifacts')
    
    # Verificar si existe la carpeta artifacts
    if artifacts_dir.exists() and any(artifacts_dir.iterdir()):
        print(f"Haciendo backup de artifacts...")
        print(f"   Origen: {artifacts_dir}")
        print(f"   Destino: {backup_dir / 'artifacts'}")
        
        # Crear directorio de backup y copiar artifacts
        backup_dir.mkdir(parents=True, exist_ok=True)
        shutil.copytree(artifacts_dir, backup_dir / 'artifacts', dirs_exist_ok=True)
        
        print(f"[OK] Backup completado: {backup_dir}")
        print(f"     Archivos copiados: {sum(1 for _ in (backup_dir / 'artifacts').rglob('*') if _.is_file())}")
        
        # Borrar contenido de artifacts (pero mantener la carpeta)
        print(f"\nLimpiando artifacts...")
        for item in artifacts_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)
            else:
                item.unlink()
        
        print(f"[OK] Artifacts limpiado. Listo para nueva ejecución.\n")
    else:
        print(f"[INFO] No hay contenido en artifacts/ para hacer backup.\n")
else:
    print(f"[INFO] Backup desactivado (HACER_BACKUP_ARTIFACTS = False)\n")

Haciendo backup de artifacts...
   Origen: artifacts
   Destino: ../../.backup_pipeline/2025-10-26_19-45-07/artifacts
[OK] Backup completado: ../../.backup_pipeline/2025-10-26_19-45-07
     Archivos copiados: 32

Limpiando artifacts...
[OK] Artifacts limpiado. Listo para nueva ejecución.



## Configuración Central de Fechas

**IMPORTANTE**: Modifica estas fechas según tus necesidades. Todos los notebooks usarán estos valores.

In [10]:
# ============================================================================
# CONFIGURACIÓN CENTRAL - MODIFICA ESTAS VARIABLES SEGÚN NECESITES
# ============================================================================

# 1. RANGO DE DATOS PARA PROCESAMIENTO (NB01, NB02)
# Controla qué datos se cargan y procesan desde las fuentes originales
# None = desde el principio
FECHA_INICIO_DATOS = None
# Última fecha de datos a procesar
FECHA_FIN_DATOS = pd.Timestamp('2025-09-20')
# 2. FECHA DE CORTE TRAIN/VALIDATION (NB03)
# Define dónde termina entrenamiento y empieza validación
FECHA_CORTE_TRAIN_VAL = pd.Timestamp('2025-09-21 00:00:00')

# 3. RANGO PARA ENTRENAMIENTO DE MODELOS (NB04, NB05, NB06, NB07)
# Permite usar solo un subconjunto del dataset de entrenamiento
# None = usar todo el dataset de train
FECHA_INICIO_ENTRENAMIENTO = pd.Timestamp('2023-01-01')
# Última fecha de entrenamiento
FECHA_FIN_ENTRENAMIENTO = pd.Timestamp('2025-09-20')

# ============================================================================

print("CONFIGURACIÓN DE FECHAS")
print("=" * 80)
print("\n1. Generación de Datos (NB01, NB02):")
print(f"   Inicio: {'Desde el principio' if FECHA_INICIO_DATOS is None else FECHA_INICIO_DATOS}")
print(f"   Fin:    {FECHA_FIN_DATOS}")

print("\n2. Separación Train/Validation (NB03):")
print(f"   Corte:  {FECHA_CORTE_TRAIN_VAL}")
print(f"   Train:  datos < {FECHA_CORTE_TRAIN_VAL}")
print(f"   Val:    datos >= {FECHA_CORTE_TRAIN_VAL}")

print("\n3. Entrenamiento de Modelos (NB04-NB07):")
print(f"   Inicio: {'Usar todo el train' if FECHA_INICIO_ENTRENAMIENTO is None else FECHA_INICIO_ENTRENAMIENTO}")
print(f"   Fin:    {FECHA_FIN_ENTRENAMIENTO}")

print("\n" + "=" * 80)

CONFIGURACIÓN DE FECHAS

1. Generación de Datos (NB01, NB02):
   Inicio: Desde el principio
   Fin:    2025-09-20 00:00:00

2. Separación Train/Validation (NB03):
   Corte:  2025-09-21 00:00:00
   Train:  datos < 2025-09-21 00:00:00
   Val:    datos >= 2025-09-21 00:00:00

3. Entrenamiento de Modelos (NB04-NB07):
   Inicio: 2023-01-01 00:00:00
   Fin:    2025-09-20 00:00:00



## Configuración de Ejecución

Define qué notebooks ejecutar y configuraciones adicionales.

In [ ]:
# ============================================================================
# CONFIGURACIÓN DE EJECUCIÓN
# ============================================================================

# Notebooks a ejecutar (marca False los que quieras saltar)
EJECUTAR_NOTEBOOKS = {
    '01_data_preparation': True,
    '02_feature_engineering': True,
    '03_exploratory_analysis': True,
    '04_baseline_models': True,
    '05_models_machine_learning': True,
    '06_hyperparameter_optimization': True,
    # Ejecutamos en Colab porque es pesado, pero podría ser True si tu máquina local puede
    '07_models_neural_networks': True,
    '08_model_comparison': True,
    '09_model_validation': True
}

# IMPORTANTE: Configuración especial para NB07 (Neural Networks)
# Este notebook es pesado y se recomienda ejecutarlo en Google Colab con GPU
# True = detener antes de NB07 para ejecución manual en Colab
EJECUTAR_NB07_EN_COLAB = True

# Directorio de notebooks
NOTEBOOKS_DIR = Path('.')

# Directorio para guardar notebooks ejecutados (con outputs)
OUTPUT_DIR = Path('artifacts/executed_notebooks')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Mostrar configuración
print("\nNOTEBOOKS A EJECUTAR:")
print("=" * 80)
for nb_name, ejecutar in EJECUTAR_NOTEBOOKS.items():
    status = "[SÍ]" if ejecutar else "[NO]"
    print(f"  {status}  {nb_name}.ipynb")

print("\n" + "=" * 80)


NOTEBOOKS A EJECUTAR:
  [SÍ]  01_data_preparation.ipynb
  [SÍ]  02_feature_engineering.ipynb
  [SÍ]  03_exploratory_analysis.ipynb
  [SÍ]  04_baseline_models.ipynb
  [SÍ]  05_models_machine_learning.ipynb
  [SÍ]  06_hyperparameter_optimization.ipynb
  [SÍ]  07_models_neural_networks.ipynb
  [SÍ]  08_model_comparison.ipynb
  [SÍ]  09_model_validation.ipynb
  [NO]  10_production_validation.ipynb



## Verificación de Dependencias

Verifica que papermill esté instalado para ejecutar notebooks programáticamente.

In [12]:
# Verificar que papermill está instalado
try:
    import papermill as pm
    print("[OK] Papermill instalado correctamente")
    print(f"  Versión: {pm.__version__}")
except ImportError:
    print("[ERROR] Papermill no está instalado")
    print("\nPara instalar papermill, ejecuta:")
    print("  pip install papermill")
    print("\nO descomenta la siguiente línea:")
    # !pip install papermill

[OK] Papermill instalado correctamente
  Versión: 2.6.0


## Función de Ejecución de Notebooks

Función helper que ejecuta un notebook con los parámetros configurados.

In [13]:
def ejecutar_notebook(notebook_name, parameters=None):
    """
    Ejecuta un notebook con papermill.
    
    Args:
        notebook_name: Nombre del notebook sin extensión
        parameters: Diccionario con parámetros a inyectar
    
    Returns:
        tuple: (bool éxito, str mensaje_error)
    """
    input_path = NOTEBOOKS_DIR / f"{notebook_name}.ipynb"
    
    if not input_path.exists():
        error_msg = f"No se encontró {input_path}"
        print(f"    [ERROR] {error_msg}")
        return False, error_msg
    
    try:
        print(f"    Ejecutando...")
        start_time = datetime.now()
        
        # Construir paths directamente como literales para evitar caché de papermill
        pm.execute_notebook(
            input_path=f"./{notebook_name}.ipynb",
            output_path=f"artifacts/executed_notebooks/{notebook_name}_executed.ipynb",
            parameters=parameters or {},
            kernel_name='python3',
            progress_bar=False
        )
        
        output_str = f"artifacts/executed_notebooks/{notebook_name}_executed.ipynb"
        
        duration = (datetime.now() - start_time).total_seconds()
        print(f"    [OK] Completado en {duration:.1f}s")
        print(f"    Output guardado: {output_str}")
        return True, None
        
    except Exception as e:
        error_msg = str(e)
        print(f"    [ERROR] Falló la ejecución")
        print(f"    Tipo de error: {type(e).__name__}")
        print(f"    Mensaje: {error_msg}")
        
        # Recalcular output_path para logging
        output_path = OUTPUT_DIR / f"{notebook_name}_executed.ipynb"
        print(f"    Notebook con error guardado en: {output_path}")
        
        # Guardar información del error en un archivo de log
        error_log_path = OUTPUT_DIR / f"{notebook_name}_ERROR.txt"
        with open(error_log_path, 'w', encoding='utf-8') as f:
            f.write(f"ERROR EN: {notebook_name}.ipynb\n")
            f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Tipo: {type(e).__name__}\n")
            f.write(f"Mensaje: {error_msg}\n")
            f.write(f"\n{'='*80}\n")
            f.write(f"DETALLES COMPLETOS DEL ERROR:\n")
            f.write(f"{'='*80}\n")
            import traceback
            f.write(traceback.format_exc())
        
        print(f"    Detalles del error guardados en: {error_log_path}")
        
        return False, error_msg

## Ejecución del Pipeline Completo

Ejecuta todos los notebooks en orden con los parámetros configurados.

In [ ]:
# Función helper para convertir Timestamps a strings (papermill necesita tipos JSON serializables)
def convertir_parametros(params):
    """Convierte pd.Timestamp a string para que sean serializables por papermill"""
    converted = {}
    for key, value in params.items():
        if isinstance(value, pd.Timestamp):
            converted[key] = value.isoformat()
        elif value is None:
            converted[key] = None
        else:
            converted[key] = value
    return converted

# Preparar parámetros para cada grupo de notebooks
parametros_nb01_nb02 = convertir_parametros({
    'FECHA_INICIO_DATOS': FECHA_INICIO_DATOS,
    'FECHA_FIN_DATOS': FECHA_FIN_DATOS
})

parametros_nb03 = convertir_parametros({
    'FECHA_CORTE_TRAIN_VAL': FECHA_CORTE_TRAIN_VAL
})

parametros_nb04_nb07 = convertir_parametros({
    'FECHA_INICIO_ENTRENAMIENTO': FECHA_INICIO_ENTRENAMIENTO,
    'FECHA_FIN_ENTRENAMIENTO': FECHA_FIN_ENTRENAMIENTO
})

# Mapeo de notebooks a parámetros
notebooks_config = [
    ('01_data_preparation', parametros_nb01_nb02),
    ('02_feature_engineering', parametros_nb01_nb02),
    ('03_exploratory_analysis', parametros_nb03),
    ('04_baseline_models', parametros_nb04_nb07),
    ('05_models_machine_learning', parametros_nb04_nb07),
    ('06_hyperparameter_optimization', parametros_nb04_nb07),
    ('07_models_neural_networks', parametros_nb04_nb07),
    ('08_model_comparison', {}),
    ('09_model_validation', {})
]

# Ejecutar pipeline
print("\n" + "=" * 80)
print("INICIANDO EJECUCIÓN DEL PIPELINE")
print("=" * 80)

resultados = {}
inicio_pipeline = datetime.now()

for nb_name, parametros in notebooks_config:
    # Verificar si llegamos al NB07 y está configurado para ejecutarse en Colab
    if nb_name == '07_models_neural_networks' and EJECUTAR_NB07_EN_COLAB:
        print("\n" + "=" * 80)
        print("[PAUSA] PAUSA EN EL PIPELINE - NB07 REQUIERE GOOGLE COLAB")
        print("=" * 80)
        
        print("\nINSTRUCCIONES PARA EJECUTAR NB07 EN GOOGLE COLAB:")
        print("-" * 80)
        
        print("\n[1] PREPARAR ARCHIVOS PARA COLAB:")
        print("   Sube los siguientes archivos a tu Google Drive manteniendo la estructura completa:")
        print("   TFM/models/demand_forecast/")
        print("   ")
        print("   - artifacts/data/train_models/")
        print("      └── features_train.parquet")
        print("   ")
        print("   - artifacts/analysis/")
        print("      └── selected_features.json")
        print("   ")
        print("   - artifacts/trained_models/")
        print("      └── tree_models_results.csv")
        
        print("\n[2] ABRIR NB07 EN GOOGLE COLAB:")
        print("   • Ve a https://colab.research.google.com")
        print("   • File > Upload notebook")
        print("   • Selecciona: 07_models_neural_networks.ipynb")
        
        print("\n[3] CONFIGURAR GPU EN COLAB:")
        print("   • Runtime > Change runtime type")
        print("   • Hardware accelerator: GPU (T4 recomendado)")
        print("   • Save")
        
        print("\n[4] EJECUTAR NB07 EN COLAB:")
        print("   • La celda 3 montará Google Drive (autoriza el acceso)")
        print("   • Ejecuta todas las celdas secuencialmente")
        print("   • Al final, los resultados se copiarán automáticamente a Google Drive")
        
        print("\n[5] RECUPERAR RESULTADOS:")
        print("   Descarga de Google Drive manteniendo la estructura completa:")
        print("   TFM/models/demand_forecast/")
        print("   ")
        print("   - artifacts/trained_models/")
        print("      ├── neural_models_results.csv")
        print("      ├── mlp_mejorado_pipeline.pkl")
        print("      ├── lstm_mejorado_pipeline.pkl")
        print("      └── cnn_lstm_mejorado_pipeline.pkl")
        print("   ")
        print("   - artifacts/data/predictions/ (opcional)")
        print("      └── neural_*.parquet")
        
        print("\n[6] CONTINUAR PIPELINE:")
        print("   Los archivos descargados ya estarán en las ubicaciones correctas.")
        print("   Luego ejecuta la siguiente celda para continuar con NB08-10")
        
        print("\n" + "=" * 80)
        print("[PAUSADO] PIPELINE PAUSADO - Esperando ejecución de NB07 en Colab")
        print("=" * 80)
        
        resultados[nb_name] = 'pendiente_colab'
        
        # Detener ejecución aquí
        print("\n[ACCION REQUERIDA] Para continuar:")
        print("   1. Ejecuta NB07 en Google Colab siguiendo las instrucciones")
        print("   2. Descarga y copia los resultados a artifacts/")
        print("   3. Ejecuta la celda de 'CONTINUACIÓN DEL PIPELINE' más abajo")
        
        # Salir del loop aquí
        break
    
    if not EJECUTAR_NOTEBOOKS.get(nb_name, False):
        print(f"\n[SALTADO] {nb_name}.ipynb")
        resultados[nb_name] = 'saltado'
        continue
    
    print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Ejecutando: {nb_name}.ipynb")
    exito, error_msg = ejecutar_notebook(nb_name, parametros)
    resultados[nb_name] = 'éxito' if exito else 'error'
    
    if not exito:
        print(f"\n" + "=" * 80)
        print(f"[ERROR CRÍTICO] EL PIPELINE SE HA DETENIDO")
        print("=" * 80)
        print(f"\nNotebook fallido: {nb_name}.ipynb")
        print(f"Razón: {error_msg}")
        print(f"\nPara revisar el error:")
        print(f"  1. Abre: artifacts/executed_notebooks/{nb_name}_executed.ipynb")
        print(f"  2. Revisa: artifacts/executed_notebooks/{nb_name}_ERROR.txt")
        print(f"\nCorrige el error y vuelve a ejecutar el pipeline.")
        print("=" * 80)
        # Detener el pipeline
        break 

    # Continuar con el siguiente notebook
    continue

print("\n" + "=" * 80)
print("PIPELINE COMPLETADO (HASTA NB06)" if '07_models_neural_networks' in resultados and resultados['07_models_neural_networks'] == 'pendiente_colab' else "PIPELINE COMPLETADO")
print("=" * 80)

duracion_total = (datetime.now() - inicio_pipeline).total_seconds()
print(f"\nTiempo total: {duracion_total / 60:.1f} minutos")

# Resumen de resultados
print("\nRESUMEN DE EJECUCIÓN:")
print("-" * 80)
exitos = sum(1 for r in resultados.values() if r == 'éxito')
errores = sum(1 for r in resultados.values() if r == 'error')
saltados = sum(1 for r in resultados.values() if r == 'saltado')
pendientes = sum(1 for r in resultados.values() if r == 'pendiente_colab')

for nb_name, resultado in resultados.items():
    if resultado == 'éxito':
        icono = "[OK]"
    elif resultado == 'error':
        icono = "[ERROR]"
    elif resultado == 'pendiente_colab':
        icono = "[PAUSADO]"
    else:
        icono = "[SALTADO]"
    print(f"  {icono} {nb_name}: {resultado}")

print("-" * 80)
print(f"Total: {exitos} éxitos, {errores} errores, {saltados} saltados, {pendientes} pendientes (Colab)")

if pendientes > 0:
    print("\n[ACCION REQUERIDA]")
    print("   Ejecuta NB07 en Google Colab y luego ejecuta la celda de CONTINUACIÓN")


INICIANDO EJECUCIÓN DEL PIPELINE

[18:49:57] Ejecutando: 01_data_preparation.ipynb
    Ejecutando...
    [OK] Completado en 2.7s
    Output guardado: artifacts/executed_notebooks/01_data_preparation_executed.ipynb

[18:50:00] Ejecutando: 02_feature_engineering.ipynb
    Ejecutando...
    [OK] Completado en 4.5s
    Output guardado: artifacts/executed_notebooks/02_feature_engineering_executed.ipynb

[18:50:04] Ejecutando: 03_exploratory_analysis.ipynb
    Ejecutando...
    [OK] Completado en 110.8s
    Output guardado: artifacts/executed_notebooks/03_exploratory_analysis_executed.ipynb

[18:51:55] Ejecutando: 04_baseline_models.ipynb
    Ejecutando...
    [OK] Completado en 3.4s
    Output guardado: artifacts/executed_notebooks/04_baseline_models_executed.ipynb

[18:51:58] Ejecutando: 05_models_machine_learning.ipynb
    Ejecutando...
    [OK] Completado en 18.2s
    Output guardado: artifacts/executed_notebooks/05_models_machine_learning_executed.ipynb

[18:52:17] Ejecutando: 06_hyper

KeyboardInterrupt: 

## Verificación de Resultados de NB07 (Google Colab)

**SOLO ejecuta esta celda después de:**
1. Haber ejecutado NB07 en Google Colab
2. Haber descargado y copiado los archivos de resultados a `artifacts/`

In [ ]:
# Verificar que los archivos necesarios de NB07 existen
print("=" * 80)
print("VERIFICACIÓN DE RESULTADOS DE NB07 (Google Colab)")
print("=" * 80)

archivos_requeridos = [
    'artifacts/trained_models/neural_models_results.csv',
    'artifacts/trained_models/mlp_mejorado_pipeline.pkl',
    'artifacts/trained_models/lstm_mejorado_pipeline.pkl',
    'artifacts/trained_models/cnn_lstm_mejorado_pipeline.pkl'
]

todos_presentes = True
print("\nVerificando archivos requeridos:")
for archivo in archivos_requeridos:
    archivo_path = Path(archivo)
    existe = archivo_path.exists()
    icono = "[OK]" if existe else "[FALTA]"
    print(f"  {icono} {archivo}")
    if not existe:
        todos_presentes = False

if todos_presentes:
    print("\n[OK] Todos los archivos de NB07 están presentes")
    print("    Puedes continuar con la siguiente celda")
    
    # Actualizar estado de resultados
    if '07_models_neural_networks' in resultados:
        resultados['07_models_neural_networks'] = 'éxito_colab'
else:
    print("\n[ERROR] FALTAN ARCHIVOS")
    print("   Por favor, completa la ejecución de NB07 en Google Colab")
    print("   y copia los archivos de resultados a artifacts/")
    
print("=" * 80)

## Continuación del Pipeline (NB08-10)

Ejecuta esta celda solo después de verificar que los archivos de NB07 están presentes.

In [ ]:
# Continuar con notebooks posteriores a NB07
if '07_models_neural_networks' not in resultados or resultados['07_models_neural_networks'] != 'éxito_colab':
    print("[ADVERTENCIA] Primero verifica los resultados de NB07 en la celda anterior")
else:
    print("\n" + "=" * 80)
    print("CONTINUANDO PIPELINE - NOTEBOOKS 08-10")
    print("=" * 80)
    
    # Notebooks post-NB07
    notebooks_post_colab = [
        ('08_model_comparison', {}),
        ('09_model_validation', {}),
        ('10_production_validation', {}),
    ]
    
    for nb_name, parametros in notebooks_post_colab:
        if not EJECUTAR_NOTEBOOKS.get(nb_name, False):
            print(f"\n[SALTADO] {nb_name}.ipynb")
            resultados[nb_name] = 'saltado'
            continue
        
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Ejecutando: {nb_name}.ipynb")
        exito, error_msg = ejecutar_notebook(nb_name, parametros)
        resultados[nb_name] = 'éxito' if exito else 'error'
        
        if not exito:
            print(f"\n" + "=" * 80)
            print(f"[ERROR CRÍTICO] EL PIPELINE SE HA DETENIDO")
            print("=" * 80)
            print(f"\nNotebook fallido: {nb_name}.ipynb")
            print(f"Razón: {error_msg}")
            print(f"\nPara revisar el error:")
            print(f"  1. Abre: artifacts/executed_notebooks/{nb_name}_executed.ipynb")
            print(f"  2. Revisa: artifacts/executed_notebooks/{nb_name}_ERROR.txt")
            print(f"\nCorrige el error y vuelve a ejecutar el pipeline.")
            print("=" * 80)
            break  # Detener el pipeline
    
    print("\n" + "=" * 80)
    print("PIPELINE COMPLETO FINALIZADO")
    print("=" * 80)
    
    # Resumen final actualizado
    duracion_total = (datetime.now() - inicio_pipeline).total_seconds()
    print(f"\nTiempo total: {duracion_total / 60:.1f} minutos")
    
    print("\nRESUMEN FINAL DE EJECUCIÓN:")
    print("-" * 80)
    exitos = sum(1 for r in resultados.values() if r in ['éxito', 'éxito_colab'])
    errores = sum(1 for r in resultados.values() if r == 'error')
    saltados = sum(1 for r in resultados.values() if r == 'saltado')
    
    for nb_name, resultado in resultados.items():
        if resultado in ['éxito', 'éxito_colab']:
            icono = "[OK]"
        elif resultado == 'error':
            icono = "[ERROR]"
        else:
            icono = "[SALTADO]"
        
        # Mostrar (Colab) si fue ejecutado en Colab
        label = f"{nb_name} (Colab)" if resultado == 'éxito_colab' else nb_name
        print(f"  {icono} {label}: {resultado if resultado != 'éxito_colab' else 'éxito'}")
    
    print("-" * 80)
    print(f"Total: {exitos} éxitos, {errores} errores, {saltados} saltados")

## Resumen de Artefactos Generados

Verifica qué archivos se generaron durante el pipeline.

In [ ]:
print("\nARTEFACTOS GENERADOS:")
print("=" * 80)

artifacts_base = Path('artifacts')

# Datos procesados (NB01)
print("\n1. Datos Procesados (NB01):")
processed_dir = artifacts_base / 'data' / 'processed'
if processed_dir.exists():
    for file in sorted(processed_dir.glob('*.parquet')):
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"   [OK] {file.name} ({size_mb:.1f} MB)")
else:
    print("   [NO ENCONTRADO]")

# Features (NB02)
print("\n2. Features (NB02):")
features_dir = artifacts_base / 'data' / 'features'
if features_dir.exists():
    for file in sorted(features_dir.glob('*.parquet')):
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"   [OK] {file.name} ({size_mb:.1f} MB)")
else:
    print("   [NO ENCONTRADO]")

# Datasets Train/Val (NB03)
print("\n3. Datasets Train/Validation (NB03):")
train_dir = artifacts_base / 'data' / 'train_models'
val_dir = artifacts_base / 'data' / 'validation_models'
for dir_path, label in [(train_dir, 'Train'), (val_dir, 'Validation')]:
    if dir_path.exists():
        for file in sorted(dir_path.glob('*.parquet')):
            size_mb = file.stat().st_size / (1024 * 1024)
            print(f"   [OK] {label}: {file.name} ({size_mb:.1f} MB)")

# Modelos entrenados (NB04-NB07)
print("\n4. Modelos Entrenados (NB04-NB07):")
models_dir = artifacts_base / 'trained_models'
if models_dir.exists():
    # Archivos CSV de resultados
    for file in sorted(models_dir.glob('*_results.csv')):
        print(f"   [OK] {file.name}")
    # Pipelines de modelos
    pipelines = list(models_dir.glob('*_pipeline.pkl'))
    if pipelines:
        print(f"   [OK] {len(pipelines)} pipelines guardados")
else:
    print("   [NO ENCONTRADO]")

# Análisis y recomendaciones (NB03, NB08)
print("\n5. Análisis y Recomendaciones:")
analysis_dir = artifacts_base / 'analysis'
if analysis_dir.exists():
    for file in sorted(analysis_dir.glob('*.json')):
        print(f"   [OK] {file.name}")
else:
    print("   [NO ENCONTRADO]")

# Optuna (NB06, opcional)
print("\n6. Optimización Optuna (NB06, opcional):")
optuna_dir = artifacts_base / 'optuna'
if optuna_dir.exists():
    for file in sorted(optuna_dir.glob('*.json')):
        print(f"   [OK] {file.name}")
else:
    print("   [OPCIONAL - NO EJECUTADO]")

print("\n" + "=" * 80)

## Modelo Recomendado

Muestra el modelo ganador seleccionado por el pipeline.

In [ ]:
# Cargar recomendación del modelo
recommendation_path = Path('artifacts/analysis/model_recommendation.json')

if recommendation_path.exists():
    with open(recommendation_path, 'r') as f:
        recommendation = json.load(f)
    
    print("\nMODELO RECOMENDADO PARA PRODUCCIÓN:")
    print("=" * 80)
    
    # Soportar ambos formatos (español e inglés)
    modelo = recommendation.get('modelo_recomendado') or recommendation.get('best_model')
    metricas = recommendation.get('metricas') or recommendation.get('metrics')
    
    print(f"\n[GANADOR] Modelo ganador: {modelo}")
    print("\nMétricas en Test Set:")
    
    if metricas:
        for metric, value in metricas.items():
            if metric == 'tiempo_entrenamiento' or metric == 'training_time':
                print(f"  • Tiempo entrenamiento: {value:.2f}s")
            elif metric in ['mae', 'rmse']:
                print(f"  • {metric.upper()}: {value:.2f} MW")
            elif metric == 'mape':
                print(f"  • {metric.upper()}: {value:.2f}%")
            elif metric == 'r2':
                print(f"  • R²: {value:.4f}")
    
    # Mostrar comparación de enfoques si está disponible
    if 'comparacion_enfoques' in recommendation:
        comp = recommendation['comparacion_enfoques']
        print("\nComparación de Enfoques:")
        print(f"  • Manual (NB05):        MAE = {comp['manual_mae']:.2f} MW")
        print(f"  • Optuna (NB06):        MAE = {comp.get('optuna_mae', 'N/A')}")
        print(f"  • Neural Networks (NB07): MAE = {comp['neural_mae']:.2f} MW")
    
    print("\n" + "=" * 80)
else:
    print("\n[ADVERTENCIA] No se encontró recomendación de modelo.")
    print("   Ejecuta el NB08 (model_comparison) para seleccionar el mejor modelo.")